In [86]:
import numpy as np 
import pandas as pd
import csv 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import os 
os.chdir("/Users/susanchen/Documents/GitHub/skincare_classification/Plots")

## Building Multi-label Classifiers 
### Classifiers to test: Binary Relevance (no correlation), Binary Relevance (with correlation), Multi-label Random Forest, Multi-label Decision Tree, Classifier Chain, Binary Releveance with Stacking Aggregation, Cross-Coupling Aggregation (to combat class-imbalances)

In [89]:
data = pd.read_csv('/Users/susanchen/Documents/GitHub/skincare_classification/Data_sets/Master_cleaned.csv')

In [90]:
data

,Product,Brand,Ingredients,Price,Skin_Type,active_ingredient,inactive_ingredient,active_ingredient_list,inactive_ingredient_list,num_inactive_ingredients,...,num_of_Emollients,num_of_Hydration,num_of_Skin-Restoring,num_of_Plant Extracts,num_of_Preservatives,num_of_Skin-Softening,num_of_Sensitizing,num_of_Skin-Replenishing,top_3,top3_category_list
0,#InstantDetox Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Trehalose, H...",3.99,['Oily'],NaN,"Water, Butylene Glycol, Glycerin, Trehalose, H...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Treh...",23,...,0,0,0,0,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Trehalose']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
1,#Peachy Facial Mask,Biobelle,"Water, Methylpropanediol, Butylene Glycol, Gly...",3.99,['Dry'],NaN,"Water, Methylpropanediol, Butylene Glycol, Gly...",[],"['Water', 'Methylpropanediol', 'Butylene Glyco...",20,...,0,0,0,0,0,0,0,1,"['Methylpropanediol', 'Butylene Glycol', 'Glyc...","[None, 'Texture Enhancer', 'Skin-Replenishing,..."
2,#Rise&Shine Facial Mask,Biobelle,"Water, Glycerin, Butylene Glycol, Triethylhexa...",3.99,['Combination'],NaN,"Water, Glycerin, Butylene Glycol, Triethylhexa...",[],"['Water', 'Glycerin', 'Butylene Glycol', 'Trie...",33,...,0,0,0,0,0,0,0,1,"['Glycerin', 'Butylene Glycol', 'Triethylhexan...","['Skin-Replenishing, Skin-Restoring', 'Texture..."
3,#RoséAllDay Facial Mask,Biobelle,"Water, Methylpropanediol, Glycerin, Propanedio...",3.99,['Combination'],NaN,"Water, Methylpropanediol, Glycerin, Propanedio...",[],"['Water', 'Methylpropanediol', 'Glycerin', 'Pr...",31,...,0,0,0,1,2,0,0,2,"['Methylpropanediol', 'Glycerin', 'Propanediol']","[None, 'Skin-Replenishing, Skin-Restoring', None]"
4,#VitaminSea Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",3.99,['Dry'],NaN,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Hydr...",24,...,0,0,0,1,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Hydroxyacetop...","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Youthful Vitamin C Fresh Radiance Essence,No7,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",24.99,"['Combination', 'Dry', 'Normal', 'Oily']",NaN,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",[],"['Aqua (Water)', 'Butylene Glycol', 'Glycerin'...",18,...,0,0,0,2,3,0,0,2,"['Butylene Glycol', 'Glycerin', 'Gluconolactone']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
2020,Yuza Sorbet Day Cream,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",48.00,"['Combination', 'Dry', 'Normal', 'Oily', 'Sens...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",33,...,1,0,0,1,1,0,0,1,"['Cyclomethicone', 'Glycerin', 'Nylon-12']","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2021,Yuza Sorbet Night Treatment,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",55.00,"['Combination', 'Dry', 'Normal', 'Oily', 'Sens...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",15,...,2,0,0,0,0,0,0,1,"['Cyclomethicone', 'Glycerin', 'Cetearyl Alcoh...","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2022,Yuzu Overnight Moisture Mask,Earth Therapeutics,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",7.00,"['Combination', 'Dry', 'Normal', 'Sensitive']",NaN,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",[],"['Water (Aqua)', 'Propanediol', 'Glycerin', 'H...",41,...,2,0,0,1,0,0,0,1,"['Propanediol', 'Glycerin', 'Hydrogenated Poly...","[None, 'Skin-Replenishing, Skin-Restoring', None]"


In [151]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

X = data[data.columns[14:23]].values 
y = data.Skin_Type
y = MultiLabelBinarizer().fit_transform(y)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, shuffle=True, test_size=0.4, random_state = 1024)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, shuffle=False, test_size=0.5, random_state = 1024)

print("Dataset sizes:\n\tTrain %s\n\tVal %s\n\tTest %s" % (X_train.shape, X_val.shape,X_test.shape))

Dataset sizes:
	Train (1214, 9)
	Val (405, 9)
	Test (405, 9)


## Binary Relevance with Gaussian 

In [213]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
# initialize binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(
    classifier = GaussianNB(),
    require_dense=[True, True] 
)
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_val)
# accuracy
print("F1 Score = ", metrics.f1_score(y_val,predictions, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, predictions))


F1 Score =  0.8569786010859151
Hamming Loss 0.24036500268384325


## One vs the Rest Classifier 

In [215]:
from sklearn.multiclass import OneVsRestClassifier
classif = OneVsRestClassifier(SVC(kernel='linear', random_state= 1024))
classif.fit(X_train, y_train)
val_pred = classif.predict(X_val)
# accuracy
print("F1 Score = ", metrics.f1_score(y_val,val_pred, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, val_pred))

F1 Score =  0.9013751991973086
Hamming Loss 0.1793880837359098


## Decision Tree Classifier

In [175]:
from sklearn.model_selection import GridSearchCV
dt =  DecisionTreeClassifier()
param_grid = {
    'max_depth' :[5,6,7,8,9],
    'max_leaf_nodes': [5,10,15]
}
gs = GridSearchCV(dt, param_grid, scoring = "accuracy", cv= 5)
gs.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_leaf_nodes': [5, 10, 15]},
             scoring='accuracy')

In [217]:
tree = dt.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(metrics.f1_score(y_test, y_pred, average="micro"))

0.8535602910602912


  ## Binary Relevance with Random Forest Classifier 

In [205]:
from sklearn.ensemble import RandomForestClassifier
start=time.time()
classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')


training time taken:  8.0 seconds


In [208]:
y_hat=classifier.predict(X_test)
br_f1=metrics.f1_score(y_test, y_hat, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_hat)
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

Binary Relevance F1-score: 0.889
Binary Relevance Hamming Loss: 0.192


## Label Powerset with Random Forest Classifier

In [209]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

start=time.time()
classifier.fit(X_train, y_train)
print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  1.0 seconds


In [211]:
y_hat=classifier.predict(X_test)
lp_f1=metrics.f1_score(y_test, y_hat, average='micro')
lp_hamm=metrics.hamming_loss(y_test,y_hat)
print('Label Powerset F1-score:',round(lp_f1,3))
print('Label Powerset Hamming Loss:',round(lp_hamm,3))

Label Powerset F1-score: 0.886
Label Powerset Hamming Loss: 0.199


## Multi-label K Nearest Neighbors

In [180]:
from skmultilearn.adapt import MLkNN
import time

parameters = {'k': range(1,3), 
              's': [0.5, 0.7, 1.0]}

score = 'f1_micro'

start=time.time()

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_, 'best score: ',
      classifier.best_score_)

training time taken:  37.0 seconds
best parameters : {'k': 2, 's': 0.5} best score:  0.8925032018569974


## 

## Binary Relevance K-Nearest Neighbors

In [218]:
from skmultilearn.adapt import BRkNNaClassifier

parameters = {'k': range(3,5)}
score = 'f1_samples'

start=time.time()

classifier = GridSearchCV(BRkNNaClassifier(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_,
      'best score: ',classifier.best_score_)


training time taken:  1.0 seconds
best parameters : {'k': 3} best score:  0.8506303900626543


In [202]:
BrKnn = BRkNNaClassifier(k= 3)
BrKnn.fit(X_train, y_train)
pred = BrKnn.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.2641975308641975
